# LGMVIP - Task 2 (Advanced Level)
# Next Word Prediction

In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [2]:
text = open('C:/Users/Hrishikesh Choraghe/Desktop/LGMVIP/NextWordPrediction/1661-0.txt',encoding = 'utf-8').read().lower()
print('corpus length:', len(text))

corpus length: 581888


In [3]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [4]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [5]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


In [6]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)

In [7]:
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [8]:
print(X[0][0])

[False False False ... False False False]


In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [10]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

Epoch 1/20


C:\Users\Hrishikesh Choraghe\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


811/811 [==============================] - 89s 108ms/step - loss: 6.0134 - accuracy: 0.1069 - val_loss: 7.0265 - val_accuracy: 0.0987
Epoch 2/20
811/811 [==============================] - 89s 110ms/step - loss: 5.7783 - accuracy: 0.1480 - val_loss: 8.0090 - val_accuracy: 0.1102
Epoch 3/20
811/811 [==============================] - 88s 108ms/step - loss: 5.7483 - accuracy: 0.1767 - val_loss: 8.0541 - val_accuracy: 0.1095
Epoch 4/20
811/811 [==============================] - 88s 109ms/step - loss: 5.4323 - accuracy: 0.2127 - val_loss: 8.3282 - val_accuracy: 0.1060
Epoch 5/20
811/811 [==============================] - 87s 108ms/step - loss: 5.1115 - accuracy: 0.2513 - val_loss: 8.6017 - val_accuracy: 0.0943
Epoch 6/20
811/811 [==============================] - 86s 106ms/step - loss: 4.8188 - accuracy: 0.2915 - val_loss: 8.5957 - val_accuracy: 0.0910
Epoch 7/20
811/811 [==============================] - 88s 108ms/step - loss: 4.5710 - accuracy: 0.3315 - val_loss: 8.8956 - val_accuracy: 0.0

In [11]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))

model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [12]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("It is not a lack".lower())

it
is
not
a
lack


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [13]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [14]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [15]:
q =  "Your life will never be there in the same situation again"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  Your life will never be there in the same situation again
Sequence:  your life will never be
your
life
will
never
be
next possible words:  ['my', 'time', 'able', 'much', 'better']
